In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../buckets/b1/exp/TS5410/dataset_training.csv.gz')

In [3]:
df['clase_ternaria'].replace({'CONTINUA':0, 'BAJA+2':1, 'BAJA+1':1}, inplace=True)

In [4]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

def undersample_majority(df, target_column):
    rus = RandomUnderSampler(random_state=42)
    X_undersampled, y_undersampled = rus.fit_resample(df.drop(target_column, axis=1), df[target_column])
    df_undersampled = pd.DataFrame(X_undersampled, columns=df.columns.drop(target_column))
    df_undersampled[target_column] = y_undersampled
    return df_undersampled



In [5]:
df_undersampled = undersample_majority(df, 'clase_ternaria')

train_data = df_undersampled[df_undersampled['fold_test'] !=1]
test_data = df_undersampled[df_undersampled['fold_test'] ==1]

/tmp/ipykernel_7826/3421876007.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_undersampled[target_column] = y_undersampled


In [ ]:
# train_data = df[df['fold_test'] !=1]
# test_data = df[df['fold_test'] ==1]

In [6]:
X = train_data.drop(columns='clase_ternaria')
y = train_data['clase_ternaria']

In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

rf_model = RandomForestClassifier()
lgbm_model = LGBMClassifier()
et_model = ExtraTreesClassifier()

#rf_model.fit(X_imputed, y)
lgbm_model.fit(X_imputed, y)
et_model.fit(X_imputed, y)

#importances_rf = rf_model.feature_importances_
importances_lgbm = lgbm_model.feature_importances_
importances_et = et_model.feature_importances_

In [ ]:
importances_mean = (importances_rf + importances_lgbm + importances_et) / 3

In [10]:
feature_importances_df = pd.DataFrame({'feature': X_imputed.columns, 'importance': importances_lgbm})
feature_importances_df.sort_values('importance', ascending=False, inplace=True)

In [25]:
X.columns = feature_importances_df['feature'].values

In [51]:
## datasets

dataset_1 = X.iloc[:,:int(X.shape[1]*0.3)]
dataset_2 = X.iloc[:,int(X.shape[1]*0.6):]
dataset_3 = X[random_values]

In [57]:
import random
random_values = random.sample(X.columns.to_list(), k=int(X.shape[1]*0.3))

,cliente_antiguedad_lag1,cpagodeservicios_lag1,mforex_sell_delta1,matm_delta1,catm_trx_delta1,mplazo_fijo_pesos_tend6,mtransferencias_recibidas_tend6,ctransferencias_recibidas,Master_fechaalta_delta1,mcheques_emitidos_delta1,...,ctarjeta_master_transacciones_tend6,cpayroll2_trx_tend6,Visa_msaldototal_tend6,mrentabilidad_delta1,Visa_madelantodolares_lag1,mpagodeservicios_lag1,mcuentas_saldo_delta1,Master_Fvencimiento_lag1,Master_msaldopesos_lag1,mpagodeservicios_delta1
0,0.0,-5.0,5.0,NaN,151.005478,7.097641e+04,0.000000,0.0,0.0,0.000000,...,NaN,0.0,0.000000,0.0,0.0,0.0,74712.531451,0,0.0,-22.094565
1,1.0,-5.0,5.0,NaN,-826.493562,7.009185e+04,259.599436,0.0,0.0,0.000000,...,NaN,-7.0,0.000000,0.0,0.0,0.0,15542.747159,0,0.0,7.077223
2,1.0,-4.0,5.0,NaN,1972.744140,1.221438e+06,0.000000,0.0,0.0,0.000000,...,NaN,0.0,33812.380520,0.0,0.0,0.0,174231.120000,0,0.0,247.862039
3,0.0,4.0,-3.0,-104548.696115,-4972.409169,2.700581e+05,0.000000,0.0,-2.0,0.000000,...,NaN,0.0,106796.768416,0.0,0.0,0.0,663277.113205,0,0.0,-303.191296
4,0.0,3.0,0.0,-1297.997179,-605.784226,4.735936e+04,0.000000,-2.0,0.0,0.000000,...,0.0,0.0,30373.133990,0.0,0.0,0.0,1432.004396,0,0.0,263.891340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34393,0.0,-5.0,5.0,NaN,767.387626,1.419528e+05,0.000000,-1.0,0.0,0.000000,...,NaN,0.0,0.000000,0.0,NaN,NaN,0.000000,0,0.0,0.000000
34394,0.0,-5.0,5.0,NaN,-31.634523,1.622318e+05,0.000000,0.0,0.0,-9281.718726,...,NaN,0.0,0.000000,0.0,NaN,NaN,0.000000,0,0.0,0.000000
34395,0.0,-5.0,5.0,NaN,-533.614509,1.622318e+05,0.000000,0.0,0.0,0.000000,...,NaN,0.0,0.000000,0.0,NaN,NaN,1288.565912,0,0.0,-41.626877
34396,1.0,-5.0,5.0,NaN,0.352670,1.419528e+05,0.000000,0.0,0.0,0.000000,...,NaN,0.0,0.000000,0.0,NaN,NaN,93.884146,0,0.0,0.000000


In [48]:
X.iloc[:,:int(X.shape[1]*0.3)]

,azar,mprestamos_personales,mtarjeta_visa_consumo,mcuentas_saldo,mcomisiones_mantenimiento_tend6,mpayroll,ctrx_quarter,ccomisiones_mantenimiento_tend6,mactivos_margen,cpayroll_trx,...,Master_mpagado_delta1,cprestamos_personales,Master_mpagosdolares_delta1,Visa_Fvencimiento_lag1,cplazo_fijo,ctransferencias_emitidas_tend6,Visa_Fvencimiento,cproductos_lag1,Visa_msaldopesos_delta1,Visa_mpagado_delta1
0,173678003,202106,1,0,0,28,47,2216.954460,20068.308553,366.300242,...,39482.787759,1.0,34.0,50919.893734,1.0,5.0,7394.831682,1.0,0.0,0.000000
1,75647509,202009,1,0,1,39,89,-637.490345,24936.705737,475.047049,...,307060.280278,1.0,12.0,11694.037996,1.0,13.0,15843.952572,1.0,0.0,0.000000
2,63087372,202012,1,0,0,49,233,4266.360000,42224.320000,2188.470000,...,79570.793640,2.0,4.0,10898.859630,1.0,8.0,10534.457830,1.0,0.0,0.000000
3,118664040,202103,1,0,0,72,128,10818.778922,88476.041685,14.549165,...,905236.949005,4.0,0.0,0.000000,1.0,24.0,84973.032774,1.0,0.0,0.000000
4,63605546,202010,1,0,0,56,230,2809.043851,28290.933440,2101.442338,...,12330.829348,1.0,0.0,0.000000,1.0,2.0,4171.428752,1.0,3.0,4656.556581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34393,211545769,202106,1,0,0,30,5,2078.545377,7333.384316,0.000000,...,-271.173288,1.0,0.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.000000
34394,211570670,202106,0,0,0,64,5,596.960704,2504.937876,0.000000,...,40737.816220,1.0,0.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.000000
34395,211654640,202106,1,0,0,30,5,36.417124,1204.270274,16.567791,...,20072.929671,2.0,12.0,19440.579838,1.0,0.0,0.000000,1.0,0.0,0.000000
34396,213115835,202106,1,0,0,57,2,1.400659,2.425140,0.000000,...,96.794555,2.0,0.0,0.000000,1.0,0.0,0.000000,1.0,0.0,0.000000


In [45]:
X.iloc[:3,1:2]

,mprestamos_personales
0,202106
1,202009
2,202012


In [42]:
dataset_2.iloc[:2,:]

IndexingError: Too many indexers

In [ ]:
X_sorted = feature_importance_sort(X, y)

In [ ]:
X_sorted